![@mikegchambers](../images/header.png)

# Making Inference on a SageMaker Endpoint

In this notebook, we look at a couple of different ways to make inference on SageMaker endpoints.

Before running this notebook, you should have already created and have a running an endpoint to test.

## Test Images

To help with testing we have some images to test with.  If the endpoint does not use images, or if you have alternative images to test with feel free to use other data. 

 <table style="width:100%">
  <tr>
    <th>File</th>
    <th>Image</th>
  </tr>
  <tr>
    <td>./test-images/plane.jpg</td>
    <td><img src="./test-images/plane.jpg" height="200px" width="200px"/></td>
  </tr>
  <tr>
    <td>./test-images/ship.jpg</td>
    <td><img src="./test-images/ship.jpg" height="200px" width="200px"/></td>
  </tr>
  <tr>
    <td>./test-images/car.jpg</td>
    <td><img src="./test-images/car.jpg" height="200px" width="200px"/></td>
  </tr>
</table>

## The Boto Python Way

First let's look at a way to use Python, using the AWS SDK Boto3, to call the SageMaker endpoint API. 

### Import Libraries

First, we import the Python libraries we need, including Boto3.

In [ ]:
import boto3
import json
from PIL import Image

Then we create a boto3 client to use.

In [ ]:
client = boto3.client('sagemaker-runtime')

### Load data and set variables

We have to define a couple of resources.  The SageMaker endpoint that we want to use (this needs to be running already, likely from a previous lesson), and some data to test with.

In [ ]:
# The name of the SageMaker endpoint running in our account.  
# Note this needs to be in the same region as this running code. 

endpointName = 'test-endpoint'

In [1]:
# We need to load the binary data of the image into a Python variable.

f = open('./test-images/car.jpg', 'rb') # opening a binary file
data = f.read()

### Call the endpoint

Here we construct the call to the endpoint and get the result which we store in a variable called `response`.

For more details, and to see other parameters that we can pass to the call, see the documentation here:
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint

In [ ]:
response = client.invoke_endpoint(
    EndpointName=endpointName,
    Body=data
)

The `response` we get from the function call includes a reference to a stream object.  We need to read this stream to get the inference data we need.

We do this with the following line: 

In [ ]:
result = json.loads(response['Body'].read().decode())

### Results

Now let's look at the results.  We get an array with the probability values for each class.

In [ ]:
result

### Making it look pretty (Optional)

We have the data, and its a little hard to see what the result is, so let's write a little code to get a better look at the data: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
labels = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
figure(num=None, figsize=(8, 5), dpi=80, facecolor='w', edgecolor='k')

plt.bar(range(10), result)
plt.xticks(range(10), labels)
plt.show()

In [ ]:
# And just to get the prediction in text:

index_of_prediction = np.argmax(result)
label_of_prediciton = labels[index_of_prediction]

print("This looks like a {}.".format(label_of_prediciton))

## The Command-Line Way

There are other ways to call the SageMaker endpoint.  Here we use the AWS CLI to call `sagemaker-runtime invoke-endpoint`. 

In [ ]:
! aws sagemaker-runtime invoke-endpoint --endpoint-name test --region us-east-1 --body fileb://./test-images/plane.jpg output.txt

In [ ]:
! cat output.txt